In [25]:
#import thư viện
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, regularizers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
import numpy as np


In [26]:
# Đường dẫn dataset
TRAIN_DIR = "../dataset_splits/train"
VALIDATION_DIR = "../dataset_splits/validation"
TEST_DIR = "../dataset_splits/test"

# Hyperparameters
IMG_SIZE = 150
BATCH_SIZE = 32
EPOCHS = 105
LEARNING_RATE = 1e-3
NUM_CLASSES = 6


In [27]:

train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Training generator
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Validation generator
validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Test generator
test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Lưu class names
class_names = list(train_generator.class_indices.keys())

STEPS_PER_EPOCH = int(np.ceil(train_generator.samples / BATCH_SIZE))
VALIDATION_STEPS = int(np.ceil(validation_generator.samples / BATCH_SIZE))
TEST_STEPS = int(np.ceil(test_generator.samples / BATCH_SIZE))

print(f"\nClasses: {list(train_generator.class_indices.keys())}")
print(f"Số lượng ảnh training: {train_generator.samples}")
print(f"Số lượng ảnh validation: {validation_generator.samples}")
print(f"Số lượng ảnh test: {test_generator.samples}")

Found 7245 images belonging to 6 classes.
Found 1355 images belonging to 6 classes.
Found 448 images belonging to 6 classes.

Classes: ['0', '1', '2', '3', '4', '5']
Số lượng ảnh training: 7245
Số lượng ảnh validation: 1355
Số lượng ảnh test: 448


In [28]:
#Xây dựng model
model = models.Sequential([
    # Block 1
    layers.Conv2D(32, (3, 3), activation='relu', 
                 input_shape=(IMG_SIZE, IMG_SIZE, 3), name='conv1'),
    layers.MaxPooling2D((2, 2), name='pool1'),
    
    # Block 2
    layers.Conv2D(64, (3, 3), activation='relu', name='conv2'),
    layers.MaxPooling2D((2, 2), name='pool2'),
    
    # Block 3
    layers.Conv2D(128, (3, 3), activation='relu', name='conv3'),
    layers.MaxPooling2D((2, 2), name='pool3'),
    
    # Block 4
    layers.Conv2D(256, (3, 3), activation='relu', name='conv4'),
    layers.MaxPooling2D((2, 2), name='pool4'),
    
    # Fully Connected Layers
    layers.Flatten(name='flatten'),
    layers.Dropout(0.6, name='dropout'),
    layers.Dense(256, activation='relu', 
                kernel_regularizer=regularizers.l2(0.002), name='fc1'),
    layers.Dense(NUM_CLASSES, activation='softmax', name='output')
])

# Hiển thị kiến trúc model
model.summary()

C:\Users\pnthv\AppData\Roaming\Python\Python313\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1 (Conv2D)                  │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool1 (MaxPooling2D)            │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2 (Conv2D)                  │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool2 (MaxPooling2D)            │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3 (Conv2D)                  │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool3 (MaxPooling2D)            │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv4 (Conv2D)                  │ (None, 15, 15, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool4 (MaxPooling2D)            │ (None, 7, 7, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 256)            │     3,211,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 6)              │         1,542 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,601,478 (13.74 MB)

 Trainable params: 3,601,478 (13.74 MB)

 Non-trainable params: 0 (0.00 B)

In [29]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizers.Adam(learning_rate=LEARNING_RATE),
    metrics=['accuracy']
)


In [30]:
#callbacks
callbacks = [
    ModelCheckpoint(
        './saved_model/tomato_best.h5',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    
    # Early stopping
    EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1
    ),
    
    # Giảm learning rate khi plateau
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    )
]


In [31]:
#training
history = model.fit(
    train_generator,
    steps_per_epoch=STEPS_PER_EPOCH,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=VALIDATION_STEPS,
    callbacks=callbacks,
    verbose=1
)

# Lưu model cuối cùng
model.save('./saved_model/tomato_final.h5')
print("\nModel đã được lưu tại './saved_model/tomato_final.h5'")

Epoch 1/105
  8/227 ━━━━━━━━━━━━━━━━━━━━ 2:31 693ms/step - accuracy: 0.2159 - loss: 2.7770

KeyboardInterrupt: 

In [ ]:
#đánh giá trên tập test
test_loss, test_acc = model.evaluate(
    test_generator,
    steps=TEST_STEPS,
    verbose=1
)

print(f"\nTest Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc*100:.2f}%")

In [ ]:
#vẽ biểu đồ quá trình training
def plot_training_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs_range = range(1, len(acc) + 1)
    
    plt.figure(figsize=(14, 5))
    
    # Subplot 1: Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, 'bo-', label='Training Accuracy', linewidth=2)
    plt.plot(epochs_range, val_acc, 'ro-', label='Validation Accuracy', linewidth=2)
    plt.title('Training and Validation Accuracy', fontsize=14, fontweight='bold')
    plt.xlabel('Epochs', fontsize=12)
    plt.ylabel('Accuracy', fontsize=12)
    plt.legend(loc='lower right')
    plt.grid(True, alpha=0.3)
    
    # Subplot 2: Loss
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, 'bo-', label='Training Loss', linewidth=2)
    plt.plot(epochs_range, val_loss, 'ro-', label='Validation Loss', linewidth=2)
    plt.title('Training and Validation Loss', fontsize=14, fontweight='bold')
    plt.xlabel('Epochs', fontsize=12)
    plt.ylabel('Loss', fontsize=12)
    plt.legend(loc='upper right')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('./results/training_history.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # In thông tin chi tiết
    print("\n" + "=" * 70)
    print("KẾT QUẢ CHI TIẾT")
    print("=" * 70)
    print(f"Độ chính xác training tốt nhất: {max(acc)*100:.2f}%")
    print(f"Độ chính xác validation tốt nhất: {max(val_acc)*100:.2f}%")
    print(f"Giá trị loss cuối cùng trên training: {loss[-1]:.4f}")
    print(f"Giá trị loss cuối cùng trên validation: {val_loss[-1]:.4f}")

# Vẽ biểu đồ
plot_training_history(history)

In [ ]:
print(f"""
THÔNG TIN MODEL:
- Kiến trúc: Custom CNN (4 Conv blocks)
- Input shape: {IMG_SIZE}x{IMG_SIZE}x3
- Số lớp phân loại: {NUM_CLASSES}
- Tổng số parameters: {model.count_params():,}

HYPERPARAMETERS:
- Learning rate: {LEARNING_RATE}
- Batch size: {BATCH_SIZE}
- Epochs: {len(history.history['loss'])} (tối đa {EPOCHS})
- Optimizer: Adam
- Loss function: Categorical Crossentropy

REGULARIZATION:
- Dropout: 0.6
- L2 regularization: 0.002
- Early stopping: patience=15
- Learning rate reduction: factor=0.5, patience=5

KẾT QUẢ:
- Training Accuracy: {history.history['accuracy'][-1]*100:.2f}%
- Validation Accuracy: {history.history['val_accuracy'][-1]*100:.2f}%
- Test Accuracy: {test_acc*100:.2f}%
- Test Loss: {test_loss:.4f}
""")


In [11]:
!git remote -v

origin	https://github.com/bbelal/PlantVillage-Project.git (fetch)
origin	https://github.com/bbelal/PlantVillage-Project.git (push)


In [15]:
!git remote set-url origin https://github.com/tano-dev/Tomato-Leaf-Diseases-Image-Classification-Using-Deep-Learning.git


In [22]:
!git remote -v


origin	https://github.com/tano-dev/Tomato-Leaf-Diseases-Image-Classification-Using-Deep-Learning.git (fetch)
origin	https://github.com/tano-dev/Tomato-Leaf-Diseases-Image-Classification-Using-Deep-Learning.git (push)


In [23]:
!git status -sb

## pntv
 M ../.ipynb_checkpoints/create_dataset_small-checkpoint.ipynb
A  convNet_1.ipynb
 D ../figures/inspiration.jpeg
?? saved_model/


In [24]:
!git push -u origin pntv

branch 'pntv' set up to track 'origin/pntv'.


remote: 
remote: Create a pull request for 'pntv' on GitHub by visiting:        
remote:      https://github.com/tano-dev/Tomato-Leaf-Diseases-Image-Classification-Using-Deep-Learning/pull/new/pntv        
remote: 
To https://github.com/tano-dev/Tomato-Leaf-Diseases-Image-Classification-Using-Deep-Learning.git
 * [new branch]      pntv -> pntv
